In [78]:
%matplotlib inline
import numpy as np
import pandas as pd
import networkx as nx
from source import get_region_df, get_graph

london_df = get_region_df('London.csv')
year_range = london_df['AADFYear'].unique()
london_df.dtypes



AADFYear                        int64
CP                              int64
Estimation_method              object
Estimation_method_detailed     object
Region                         object
LocalAuthority                 object
Road                           object
RoadCategory                   object
Easting                         int64
Northing                        int64
StartJunction                  object
EndJunction                    object
LinkLength_km                 float64
LinkLength_miles              float64
PedalCycles                     int64
Motorcycles                     int64
CarsTaxis                       int64
BusesCoaches                    int64
LightGoodsVehicles              int64
V2AxleRigidHGV                  int64
V3AxleRigidHGV                  int64
V4or5AxleRigidHGV               int64
V3or4AxleArticHGV               int64
V5AxleArticHGV                  int64
V6orMoreAxleArticHGV            int64
AllHGVs                         int64
AllMotorVehi

In [101]:
first = london_df[(london_df['Road']=='M1') & (london_df.AADFYear==2000)]
first

,AADFYear,CP,Estimation_method,Estimation_method_detailed,Region,LocalAuthority,Road,RoadCategory,Easting,Northing,...,LightGoodsVehicles,V2AxleRigidHGV,V3AxleRigidHGV,V4or5AxleRigidHGV,V3or4AxleArticHGV,V5AxleArticHGV,V6orMoreAxleArticHGV,AllHGVs,AllMotorVehicles,road_section_hash
0,2000,6000,Counted,Manual count,London,Barnet,M1,TM,522170,189100,...,5824,1019,91,256,278,593,374,2611,41922,3663000796566525181
1,2000,6001,Counted,Manual count,London,Barnet,M1,TM,520150,194300,...,8232,1508,141,395,373,608,451,3476,64588,3669629892576565081
770,2000,36000,Counted,Manual count,London,Barnet,M1,TM,522300,190660,...,2749,449,34,63,71,91,52,760,20017,3666467071301525731
1562,2000,73006,Estimated,Estimated from nearby links,London,Barnet,M1,TM,517540,194345,...,1917,377,31,74,99,143,94,818,15801,3668016882599120356
1619,2000,73618,Counted,Dependent on a neighbouring counted link,London,Harrow,M1,TM,516930,194600,...,9585,1884,154,367,494,712,469,4080,78992,3668395821991987881


In [115]:
def ngraph(df):
    """
    create a network plot from the dataframe
    """
    G = nx.from_pandas_edgelist(df)
    
def filter_split(df,col,year=2017):
    
    """
    take in DataFrame, filter the 
    """
    new_df = pd.DataFrame()
    for (col_name, col_check) in col_iter:
         new_df[year]= df[(df[col_name]==col_check) & (df.AADFYear==year)]
        
    return new_df


cycle_years = london_df[london_df.Road=='A1'].groupby('AADFYear').PedalCycles
cycle_years.mean.plot()
# cycle_years.groupby('Road').mean().plot(kind='bar')

AttributeError: 'function' object has no attribute 'plot'

In [116]:
london_df.describe()

,AADFYear,CP,Easting,Northing,LinkLength_km,LinkLength_miles,PedalCycles,Motorcycles,CarsTaxis,BusesCoaches,LightGoodsVehicles,V2AxleRigidHGV,V3AxleRigidHGV,V4or5AxleRigidHGV,V3or4AxleArticHGV,V5AxleArticHGV,V6orMoreAxleArticHGV,AllHGVs,AllMotorVehicles,road_section_hash
count,32980.000000,32980.000000,32980.000000,32980.000000,32980.00000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,32980.000000,3.298000e+04
mean,2008.491176,40254.896725,529934.172135,180092.763645,0.97089,0.602725,642.576380,873.984839,21399.727956,771.469254,3585.717162,666.575652,92.322620,124.522650,38.293602,82.097301,108.340752,1112.153578,27743.053214,4.122428e+18
std,5.188056,22988.215855,9903.107320,7646.281933,0.92589,0.575596,1030.932693,951.293816,17274.720403,715.854171,3224.185137,725.122767,157.753386,240.127412,93.922652,400.150591,495.184698,1733.923352,22024.638731,2.679138e+17
min,2000.000000,6000.000000,504000.000000,157500.000000,0.10000,0.060000,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,121.000000,3.653686e+18
25%,2004.000000,18465.000000,524000.000000,175120.000000,0.30000,0.190000,74.000000,243.000000,11602.750000,301.000000,1727.000000,269.000000,29.000000,20.000000,5.000000,4.000000,5.000000,367.000000,15347.000000,3.682986e+18
50%,2008.000000,37774.000000,530453.000000,180360.000000,0.70000,0.430000,229.000000,524.000000,16417.000000,569.000000,2607.000000,448.000000,52.000000,48.000000,13.000000,14.000000,19.000000,633.000000,21192.000000,4.276723e+18
75%,2013.000000,56977.000000,535530.000000,185000.000000,1.30000,0.810000,752.000000,1175.000000,23967.750000,1010.000000,4065.000000,748.000000,96.000000,116.000000,34.000000,43.000000,61.000000,1098.250000,30945.500000,4.283122e+18
max,2017.000000,99907.000000,558535.000000,200000.000000,8.50000,5.280000,12250.000000,9815.000000,207133.000000,7238.000000,38449.000000,10942.000000,4059.000000,3938.000000,2010.000000,11252.000000,13382.000000,27036.000000,262842.000000,4.316291e+18
